In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
from mvl import genData
from torch import tensor

liabParams55cov = genData.genParams(pis=tensor([.1, .1, .05]), rrMeans=tensor([3., 2.]), afMean = tensor(1e-4), pDs = tensor([.01, .01]), afShape=tensor(50.), nCases=tensor([1.5e4, 1.5e4, 4e3]), nCtrls=tensor(5e4), covShared=tensor([ [1, .95], [.95, 1] ]), meanEffectCovarianceScale=tensor(.01))[0]
liabParams55cov["pDs"] = liabParams55cov["pDs"][0:2]
liabData55cov = genData.v6liability(**liabParams55cov)
liabParams55cov["pDs"] = liabData55cov["PDs"] #TODO: normalized names, and indicate that the params pDs are incomplete

covShared tensor([[1.0000, 0.9500],
        [0.9500, 1.0000]])
residualCovariance tensor([[1, 0],
        [0, 1]])
PD1 threshold, PD2 threshold tensor(-2.3263) tensor(-2.3263)
pDsWithBoth tensor([0.0100, 0.0100, 0.0067], dtype=torch.float64)
pdThresh tensor([2.3263, 2.3263])
pdTarget tensor([0.0300, 0.0200])
pdvthresh tensor([1.8808, 2.0537])
meanEffect tensor([0.4456, 0.2726])
meanEffectsAcrossAllGenes tensor([0.4456, 0.2726])
allEffects tensor([[-0.3364, -0.1989],
        [-0.4003, -0.1964],
        [-0.3531, -0.1854],
        ...,
        [-0.3658, -0.2093],
        [-0.2078, -0.0512],
        [-0.4293, -0.2844]])
PD1GivenV.mean() tensor(0.0307) PD2GivenV.mean() tensor(0.0205)
allEffects[i] tensor([-0.3364, -0.1989])
PDBothGivenV.mean tensor(0.0171, dtype=torch.float64)
PDBothGivenV / PDBoth tensor(2.5588, dtype=torch.float64)
pdsCovarOnMean.mean(0) tensor([0.0307, 0.0205, 0.0171], dtype=torch.float64)
np.corrcoef(pdvInBoth[:,0], pdvInBoth[:,1])
 [[1.         0.94828208]
 [0.9482820

In [3]:
liabParams55cov["pDs"]

tensor([0.0100, 0.0100, 0.0067], dtype=torch.float64)

In [4]:
liabParams55cov["pDsample"]

KeyError: 'pDsample'

In [21]:
import numpyro
from numpyro.distributions import Multinomial, Beta, Dirichlet, Gamma, Beta, Categorical, Uniform, MultivariateNormal, Normal, LogNormal, Exponential
from jax import random
from numpyro.infer import MCMC, NUTS
import numpy as np
import jax.numpy as jnp

numpyro.set_host_device_count(6)


def mix_weights(beta: jnp.array):
    beta_cumprod = (1 - beta).cumprod(-1)
    return jnp.pad(beta, (0,1), constant_values=1) * jnp.pad(beta_cumprod, (1,0), constant_values = 1)


# mu_exp, var_exp = get_log_params(liabParams55cov["afMean"].numpy(), 1)

nHypotheses = 4
kConditions = 4
altCounts = liabData55cov["altCounts"].numpy()
N = len(liabData55cov["altCounts"])

nCases = liabParams55cov["nCases"].numpy()
nCtrls = liabParams55cov["nCtrls"].numpy()
samplePDs = nCases / (nCases.sum() + nCtrls)
pdsAll = np.array([1 - samplePDs.sum(), *samplePDs])
print("pdsAll", pdsAll)

# TODO: do this in numpy natively
pdsAllShaped = jnp.asarray(tensor(pdsAll).expand(nHypotheses, kConditions).numpy())
pdsAllnp = [1 - samplePDs.sum(), *samplePDs]
pdsAll = jnp.asarray([1 - samplePDs.sum(), *samplePDs])
print(pdsAllShaped)
print(pdsAll.shape)
print((1/pdsAll).shape)
alpha = .05
print("nHypotheses", nHypotheses)

component_af_means = jnp.array([.005, .006, .006, .013])

def model(data):
    with numpyro.plate("beta_plate", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha))

    with numpyro.plate("prob_plate", nHypotheses):
        conc = numpyro.sample('conc', Exponential(jnp.array([.001]).repeat(kConditions), 1000).to_event(1))
        probs = numpyro.sample("probs", Dirichlet(conc * pdsAll))

    with numpyro.plate("data", N):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)

def modelScaledInDirichlet(data):
    # mu = numpyro.sample('mu', Uniform(jnp.ones(kConditions), 1000))
    # print("mu", mu)
    # concentrations = numpyro.sample("concentrations", Uniform(1, 1000))
    # print("concentrations", concentrations)
    with numpyro.plate("beta_plate2", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha))

    with numpyro.plate("mu_plate", nHypotheses):
        mu2 = numpyro.sample('mu3', Uniform(jnp.ones(kConditions), 1000))
        probs = numpyro.sample("probs2", Dirichlet(mu2*pdsAll))

    with numpyro.plate("data", N):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)

def modelPDScaledDeterministic(data):
    pdsSite = numpyro.deterministic("pDs", pdsAll)
    # mu = numpyro.sample('mu', Uniform(jnp.ones(kConditions), 1000))
    # print("mu", mu)
    # concentrations = numpyro.sample("concentrations", Uniform(1, 1000))
    # print("concentrations", concentrations)
    with numpyro.plate("beta_plate2", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha))

    with numpyro.plate("mu_plate", nHypotheses):
        mu = numpyro.sample('mu', Uniform(jnp.ones(kConditions), 1000))
        probs = numpyro.sample("probs", Dirichlet(mu))
        # print("probs", probs)
        probs = numpyro.deterministic("probs_scaled", probs * pdsAll)
        # print("probs after scaling", probs)

    with numpyro.plate("data", N):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)


def modelAf(data):
    # pds = numpyro.deterministic("pds", liabParams55cov["afMean"].numpy())# numpyro.sample("af", LogNormal(mu_exp, var_exp)
    # print("af", af)
    # mu = numpyro.sample('mu', Uniform(jnp.ones(kConditions), 1000))
    # print("mu", mu)
    # concentrations = numpyro.sample("concentrations", Uniform(1, 1000))
    # print("concentrations", concentrations)
    with numpyro.plate("beta_plate2", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha))

    with numpyro.plate("mu_plate", nHypotheses):
        conc = numpyro.sample('conc', Uniform(jnp.ones(kConditions), 1000))
        # doing this speeds up MCMC by 2x
        pdconc = numpyro.deterministic("pdconc", conc * pdsAll)
        probs = numpyro.sample("probs", Dirichlet(pdconc))
        # print("probs", probs)
        # probs = numpyro.deterministic("probs_scaled", probs * af)
        # print("probs after scaling", probs)

    with numpyro.plate("data", N):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        probs = probs[z] / component_af_means[z]
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)


# def modelLogNormal(data):
#     # tau = numpyro.param('tau', MultivariateNormal(jnp.zeros(kConditions), 3 * jnp.eye(kConditions)))
#     with numpyro.plate("beta_plate", nHypotheses-1):
#         beta = numpyro.sample("beta", Beta(1, alpha))

#     with numpyro.plate("mu_plate", nHypotheses):
#         # rrMean = numpyro.param("rrMean")
#         rr = numpyro.sample("rr", LogNormal(jnp.eye(kConditions), jnp.eye(kConditions)).to_event(1))
#         af = numpyro.sample("af", LogNormal(jnp.array(liabParams55cov["afMean"]), jnp.ones(kConditions)).to_event(1))
#         pvd = rr * af
#         print("pvd", pvd)
#         pvdpv = pvd*pdsAll
#         print("pvdpv", pvdpv)
        
#         probs = numpyro.sample("probs", Dirichlet(pvdpv))
#         # just slows things down probs = numpyro.deterministic("probs_scaled", probs * pdsAll)

#     with numpyro.plate("data", N):
#         z = numpyro.sample("z", Categorical(mix_weights(beta)))
#         return numpyro.sample("obs", Multinomial(probs=pvdpv[z]), obs=data)

# def modelGamma(data):
#     with numpyro.plate("beta_plate", nHypothesis - 1):
#         beta = numpyro.sample("beta", Beta(1, alpha))

#     with numpyro.plate("mu_plate", nHypotheses):
#         rrMean = numpyro.param("rrMean")
#         rr = numpyro.sample("rr", Gamma(jnp.ones(kConditions), jnp.ones(kConditions)))
#         af = numpyro.sample("af", Gamma(liabParams55cov["afMean"], jnp.ones(kConditions)))
#         pvd = rr * liabParams55cov["afMean"]
#         pvdpv = pvd*pdsAll
        
#         probs = numpyro.sample("probs", Dirichlet(np.ones(kConditions)))
#         # just slows things down probs = numpyro.deterministic("probs_scaled", probs * pdsAll)

#     with numpyro.plate("data", N):
#         z = numpyro.sample("z", Categorical(mix_weights(beta)))
#         return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)

pdsAll [0.59523809 0.17857143 0.17857143 0.04761905]
[[0.5952381  0.17857143 0.17857143 0.04761905]
 [0.5952381  0.17857143 0.17857143 0.04761905]
 [0.5952381  0.17857143 0.17857143 0.04761905]
 [0.5952381  0.17857143 0.17857143 0.04761905]]
(4,)
(4,)
nHypotheses 4


In [14]:
np.array([.001]).repeat(4)

array([0.001, 0.001, 0.001, 0.001])

In [67]:
component_af_means[jnp.array([2,3,0,1,1,1])]

DeviceArray([0.006, 0.013, 0.005, 0.006, 0.006, 0.006], dtype=float32)

In [ ]:
MCMC?

In [ ]:
init_rng_key = random.PRNGKey(1273)
mcmc2k = MCMC(NUTS(model), 100, 2000, num_chains=6)
mcmc2k.run(init_rng_key, altCounts)
mcmc2k.print_summary()

In [10]:
# 1/3rd as fast as not having deterministic
init_rng_key = random.PRNGKey(1273)
mcmcDeterministicScaled = MCMC(NUTS(modelPDScaledDeterministic), 100, 1000)
mcmcDeterministicScaled.run(init_rng_key, altCounts)
mcmcDeterministicScaled.print_summary()

sample: 100%|██████████| 1100/1100 [02:22<00:00,  7.72it/s, 127 steps of size 2.04e-02. acc. prob=0.79]

                mean       std    median      5.0%     95.0%     n_eff     r_hat
   beta[0]      0.75      0.00      0.75      0.75      0.76    643.13      1.00
   beta[1]      0.39      0.01      0.39      0.38      0.40   1107.88      1.00
   beta[2]      0.33      0.01      0.33      0.32      0.35   1241.93      1.00
     mu[0]      6.01      2.23      5.67      2.07      8.89    773.89      1.00
     mu[1]      4.55      1.70      4.33      1.81      7.15    915.04      1.00
     mu[2]      4.55      1.68      4.31      2.14      7.37    992.70      1.00
     mu[3]      2.26      0.84      2.13      1.00      3.40    868.73      1.00
probs[0,0]      0.59      0.00      0.60      0.59      0.60   1558.28      1.00
probs[0,1]      0.18      0.00      0.18      0.18      0.18   1480.05      1.00
probs[0,2]      0.18      0.00      0.18      0.18      0.18   1381.87      1.00
prob

In [46]:
# Scaling by pDs gives absolutely no difference in concentrations or probabilities.
probsDetermin = mcmcDeterministicScaled.get_samples()["probs"]
print(probsDetermin.std(0))
print(probsDetermin.mean(0) * .005 / pdsAll)
print(probsDetermin.mean(0) * probsDetermin.sum(0) / (abs(1 - sDetermin.sum(0))))

concentrationsDetermin = mcmcDeterministicScaled.get_samples()["mu"]
print(concentrationsDetermin.mean(0))

betaDetermin = mcmcDeterministicScaled.get_samples()['beta']
print("inferred stick-breaking weights", mix_weights(betaDetermin).mean(0))

[[0.00060713 0.00047464 0.00046731 0.00025903]
 [0.00116307 0.00131483 0.00082089 0.00090392]
 [0.00127226 0.0014616  0.00150115 0.00091209]
 [0.0012394  0.00079334 0.00130552 0.00084485]]
[[0.00499792 0.00501112 0.00499053 0.00501977]
 [0.00244107 0.0136194  0.00263441 0.0135348 ]
 [0.00170523 0.00973307 0.00974178 0.010654  ]
 [0.00247003 0.00263366 0.01353689 0.013485  ]]
[[0.5959927  0.17997411 0.17923877 0.04882867]
 [0.2916071  0.48740935 0.09509686 0.12991066]
 [0.20400803 0.3486125  0.34892347 0.10247664]
 [0.29505494 0.09507006 0.4844626  0.12943646]]
[6.0091906 4.552491  4.5509014 2.2610247]
inferred stick-breaking weights [0.7540543  0.09633646 0.0501536  0.10140968]


In [18]:
# 1/3rd as fast as not having deterministic
# this runs after mcmc fits, not during
def postprocess(arg):
    print("postprocess arg", arg)
    return arg
init_rng_key = random.PRNGKey(12273)
mcmcAf = MCMC(NUTS(model), 100, 1000)
mcmcAf.run(init_rng_key, altCounts)
mcmcAf.print_summary()

sample: 100%|██████████| 1100/1100 [01:55<00:00,  9.56it/s, 127 steps of size 9.24e-03. acc. prob=0.91]

                mean       std    median      5.0%     95.0%     n_eff     r_hat
   beta[0]      0.68      0.03      0.68      0.63      0.72    434.76      1.00
   beta[1]      0.32      0.08      0.32      0.18      0.45    330.53      1.00
   beta[2]      0.55      0.08      0.55      0.42      0.68    264.39      1.00
   conc[0]     44.45     16.89     41.86     17.57     69.77    424.22      1.00
   conc[1]    254.03     96.96    240.38    106.33    409.62    396.86      1.00
   conc[2]    223.74     87.06    211.68     88.10    359.40    440.13      1.00
   conc[3]   1249.15    496.87   1182.06    428.35   1964.89    492.25      1.00
probs[0,0]      0.61      0.00      0.61      0.60      0.61    602.48      1.00
probs[0,1]      0.17      0.00      0.17      0.17      0.18    844.29      1.00
probs[0,2]      0.18      0.00      0.18      0.17      0.18    483.94      1.00
prob

[[ 1.4832474   0.4596491   1.31627   ]
 [ 0.20621197  1.4732594  -0.31457236]
 [-0.26939392  0.43624252  1.2324655 ]
 [ 0.2226545  -1.1796342   1.3259519 ]]
[[1.4842482  0.46065128 1.3172708 ]
 [0.20721684 1.4742602  0.31357554]
 [0.2683976  0.43724483 1.2334664 ]
 [0.22365902 1.1786351  1.3269527 ]]
conc [[[ 0.90735793  0.8209912   1.0164589   0.87486637]
  [-1.7849189   1.3260416  -1.6953163   1.1568938 ]
  [-2.4553943  -0.26534724 -0.1234776   0.20299847]
  [-1.5983115  -1.5649264   1.8975633   2.0371034 ]]

 [[ 0.9792002   0.89225495  1.0747207   0.8737359 ]
  [-1.8825821   1.3688161  -1.6555487   1.3695055 ]
  [-2.3543627   0.00604874 -0.15217882  0.2538693 ]
  [-1.6798353  -1.6527425   2.1019602   1.9775591 ]]

 [[ 0.34747586  0.22657269  0.4035576   0.5498697 ]
  [-1.8331704   1.1519339  -1.7378138   0.95012164]
  [-2.4064114  -0.07818928  0.0178655   0.02229958]
  [-1.6976191  -1.4928161   1.8773036   1.7513798 ]]

 ...

 [[ 4.1126595   4.711544    3.5789683   2.1999114 ]
  [-1

In [19]:
# Scaling by pDs gives absolutely no difference in concentrations or probabilities.
sAf = mcmcAf.get_samples()["probs"]
print(sAf.shape)
print(sAf.std(0))
print(sAf.mean(0) * .005 / component_af_means)
# print(sAf.mean(0) * sAf.sum(0) / (abs(1 - sAf.sum(0))))

concs = mcmcAf.get_samples()["conc"]
print("conc std", concs.std(0))
print("conc mean", concs.mean(0))
print("DIrichlet version", Dirichlet(concs.mean(0)).mean)

betaScaledInDirichlet = mcmcAf.get_samples()['beta']
print("inferred stick-breaking weights", mix_weights(betaScaledInDirichlet).mean(0))

(1000, 4, 4)
[[0.00389873 0.0027325  0.00352119 0.00128708]
 [0.01540878 0.01959488 0.01955234 0.00719198]
 [0.01204855 0.01008762 0.01211935 0.00465437]
 [0.01941296 0.01230187 0.01632371 0.00668399]]
[[0.6072516  0.14409247 0.14585802 0.01723374]
 [0.361474   0.26773974 0.1974014  0.03090641]
 [0.40910125 0.31532937 0.0943295  0.03819543]
 [0.47252104 0.10365321 0.26697892 0.03181551]]
conc std [ 16.885881  96.90665   87.0161   496.62573 ]
conc mean [  44.450848  254.02948   223.74112  1249.1538  ]
DIrichlet version [0.02509398 0.14340806 0.12630928 0.7051887 ]
inferred stick-breaking weights [0.6756534  0.10423639 0.12163003 0.10109889]


In [57]:
probs_res = sAf.mean(0)
t = probs_res * liabParams55cov["afMean"].numpy()
t / t.sum()

DeviceArray([[0.14879583, 0.0447355 , 0.04464875, 0.01181993],
             [0.04983375, 0.08717146, 0.08687804, 0.02611676],
             [0.07298264, 0.02362582, 0.12108456, 0.03230699],
             [0.07355864, 0.12022852, 0.02374064, 0.0324722 ]],            dtype=float32)

In [51]:
(1 - liabData55cov["afs"].mean(0).sum())

tensor(0.9994, dtype=torch.float64)

In [38]:
# this is the marginal af, the af for each site, anything bigge than this is too big
empiricalAf = liabData55cov["altCounts"].sum(1) / N
empiricalAf

tensor([0.0052, 0.0013, 0.0022,  ..., 0.0019, 0.0019, 0.0022],
       dtype=torch.float64)

In [87]:
sAf.mean(1)

(1000, 4, 4)

In [66]:
liabData55cov["afs"].sum(1) / (liabData55cov["altCounts"].sum(1) / N)

tensor([0.2382, 0.2398, 0.2394,  ..., 0.2440, 0.2390, 0.2395],
       dtype=torch.float64)

In [52]:
liabData55cov["afs"].sum(1).numpy()

array([0.00060744, 0.00091135, 0.00112525, ..., 0.00045133, 0.00158906,
       0.00098196])

In [18]:
(probsDetermin / pdsAll).mean(0)

DeviceArray([[0.999585  , 1.0022235 , 0.99810535, 1.0039531 ],
             [0.48821422, 2.7238812 , 0.5268821 , 2.7069602 ],
             [0.34104523, 1.946614  , 1.9483553 , 2.1308005 ],
             [0.4940066 , 0.52673197, 2.7073789 , 2.6970003 ]],            dtype=float32)

In [35]:
init_rng_key = random.PRNGKey(1273)

mcmcScaledInDirichlet = MCMC(NUTS(modelScaledInDirichlet), 100, 1000)
mcmcScaledInDirichlet.run(init_rng_key, altCounts)
mcmcScaledInDirichlet.print_summary()

sample: 100%|██████████| 1100/1100 [01:17<00:00, 14.26it/s, 63 steps of size 3.14e-02. acc. prob=0.86]

                 mean       std    median      5.0%     95.0%     n_eff     r_hat
    beta[0]      0.75      0.00      0.75      0.75      0.76    387.97      1.00
    beta[1]      0.39      0.01      0.39      0.38      0.41   1266.66      1.00
    beta[2]      0.35      0.01      0.35      0.33      0.36    879.78      1.00
     mu3[0]      9.85      3.58      9.48      4.56     15.96   1430.55      1.00
     mu3[1]     24.64      9.25     23.42      8.77     38.38   1593.00      1.00
     mu3[2]     24.48      9.00     23.75     10.63     37.60   1096.18      1.00
     mu3[3]     45.99     17.41     43.99     19.76     74.96   1103.53      1.00
probs2[0,0]      0.60      0.00      0.60      0.59      0.60   1940.81      1.00
probs2[0,1]      0.18      0.00      0.18      0.18      0.18   2099.84      1.00
probs2[0,2]      0.18      0.00      0.18      0.18      0.18   1385.83     

In [36]:
# Scaling by pDs gives absolutely no difference in concentrations or probabilities.
sScaledInDirichlet = mcmcScaledInDirichlet.get_samples()["probs2"]
print(sScaledInDirichlet.std(0))
print(sScaledInDirichlet.mean(0) * pdsAll / 1e-4)
print(sScaledInDirichlet.mean(0) * sScaledInDirichlet.sum(0) / (abs(1 - sScaledInDirichlet.sum(0))))

concentrationsScaledInDirichlet = mcmcScaledInDirichlet.get_samples()["mu3"]
print(concentrationsScaledInDirichlet.mean(0))

betaScaledInDirichlet = mcmcScaledInDirichlet.get_samples()['beta']
print("inferred stick-breaking weights", mix_weights(betaScaledInDirichlet).mean(0))

[[0.00062603 0.00051163 0.00049016 0.00027287]
 [0.0011681  0.00125639 0.00073826 0.00086068]
 [0.00122986 0.00148664 0.00150814 0.00090002]
 [0.00119917 0.00072997 0.00132679 0.00081954]]
[[3542.234     319.5054    318.39386    22.705309]
 [1737.5046    862.24603   168.69148    62.27344 ]
 [1210.2887    620.88947   619.7199     48.538235]
 [1731.6997    168.49414   865.77277    61.85004 ]]
[[0.596097   0.17992865 0.1793062  0.04870257]
 [0.29290423 0.48385984 0.09547793 0.13178192]
 [0.20433344 0.34870097 0.348046   0.10294019]
 [0.29192904 0.09536742 0.48583475 0.13089284]]
[ 9.847206 24.64132  24.483877 45.99166 ]
inferred stick-breaking weights [0.75183034 0.09818115 0.05247173 0.09948575]


In [31]:
nuts3 = NUTS(model)
init_rng_key = random.PRNGKey(1273)
mcmc3 = MCMC(nuts3, 100, 1000)
res = mcmc3.run(init_rng_key, altCounts)
mcmc3.print_summary()

sample: 100%|██████████| 1100/1100 [08:22<00:00,  2.19it/s, 511 steps of size 3.57e-03. acc. prob=0.81]

                 mean       std    median      5.0%     95.0%     n_eff     r_hat
    beta[0]      0.75      0.00      0.75      0.75      0.76    442.20      1.00
    beta[1]      0.39      0.01      0.39      0.38      0.40   1288.17      1.00
    beta[2]      0.35      0.01      0.35      0.33      0.36   1326.11      1.00
     mu3[0]      5.89      2.06      5.67      2.33      8.84    662.81      1.00
     mu3[1]      4.41      1.60      4.26      1.64      6.61    791.73      1.00
     mu3[2]      4.45      1.66      4.30      1.73      6.91    712.50      1.00
     mu3[3]      2.22      0.77      2.13      1.01      3.24    974.23      1.00
probs2[0,0]      0.60      0.00      0.60      0.59      0.60   1756.15      1.00
probs2[0,1]      0.18      0.00      0.18      0.18      0.18   1510.73      1.00
probs2[0,2]      0.18      0.00      0.18      0.18      0.18   1006.49    

In [66]:
# Scaling by pDs gives absolutely no difference in concentrations or probabilities.
s3 = mcmc3.get_samples()["probs2"]
print(s3.std(0))
print(s3.mean(0) * pdsAll / 1e-4)
print(s3.mean(0) * s3.sum(0) / (abs(1 - s3.sum(0))))

concentrations3 = mcmc3.get_samples()["mu3"]
print(concentrations3.mean(0))

beta3 = mcmc3.get_samples()['beta']
print("inferred stick-breaking weights", mix_weights(beta3).mean(0))

NameError: name 'mcmc3' is not defined

In [3]:
import numpyro
import jax.numpy as jnx
with numpyro.handlers.seed(rng_seed=0):
    x = numpyro.sample('x2', numpyro.distributions.Uniform(jnx.ones(5), 1000))  # random.PRNGKey(0) is used
    print("x", x)

x [ 73.321144  21.306526 979.7079   878.8005   165.40814 ]


In [4]:
nuts2 = NUTS(model)
init_rng_key = random.PRNGKey(1273)
mcmc2 = MCMC(nuts2, 100, 1000)
res = mcmc2.run(init_rng_key, altCounts)
mcmc2.print_summary()

sample: 100%|██████████| 1100/1100 [00:47<00:00, 23.27it/s, 31 steps of size 5.47e-02. acc. prob=0.86]


NameError: name 'mcmc' is not defined

In [27]:
# Attempt at using shared concentration
s = mcmc2.get_samples()["probs2"]
print(s.std(0))
print(s.mean(0) * pdsAll / 1e-4)
print(s.mean(0) * s.sum(0) / (abs(1 - s.sum(0))))

concentrations = mcmc2.get_samples()["mu3"]
print(concentrations.mean(0))

beta = mcmc2.get_samples()['beta']
print("inferred stick-breaking weights", mix_weights(beta).mean(0))

[[0.00060655 0.00047736 0.00045819 0.00026776]
 [0.00119851 0.00136679 0.00076345 0.00084428]
 [0.00126107 0.00149363 0.00142916 0.000911  ]
 [0.00120162 0.00076245 0.00131083 0.00086376]]
[[3542.2522    319.5185    318.4024     22.69806 ]
 [1737.6328    862.2286    168.72958    62.257706]
 [1210.7587    620.82837   619.6189     48.543816]
 [1731.5288    168.46025   865.813      61.86202 ]]
[[0.5961001  0.17993596 0.17931096 0.04868735]
 [0.29292575 0.48385006 0.09549925 0.13174888]
 [0.20441239 0.34866676 0.34798947 0.10295192]
 [0.29190028 0.09534845 0.4858573  0.130918  ]]
[5.878089  4.4862313 4.48263   2.218408 ]
inferred stick-breaking weights [0.751837   0.09819707 0.0524589  0.09950271]


In [ ]:
nuts = NUTS(modelLogNormal)
init_rng_key = random.PRNGKey(1273)
mcmc = MCMC(nuts, 100, 1000)
res = mcmc.run(init_rng_key, altCounts)
mcmc.print_summary()

In [238]:
# Attempt at using shared concentration
s = mcmc.get_samples()["probs"]
print(s.std(0))
print(s.mean(0) * pdsAll / 1e-4)
print(s.mean(0) * s.sum(0) / (abs(1 - s.sum(0))))

beta = mcmc.get_samples()['beta']
print(beta.mean(0))

[[0.00135276 0.00143652 0.0015077  0.00094736]
 [0.00116426 0.00074751 0.00128939 0.0008272 ]
 [0.00062613 0.0005143  0.00049686 0.00026963]
 [0.00121846 0.00132125 0.00071697 0.00086843]]
[[1214.8066    618.3147    619.5247     48.915443]
 [1740.3151    168.90504   865.8255     61.037106]
 [3542.816     319.43088   318.16068    22.740835]
 [1759.8707    859.7818    168.8585     61.09671 ]]
[[0.20509245 0.3472591  0.3479367  0.10373226]
 [0.29337636 0.09559751 0.4858643  0.12918577]
 [0.59619474 0.1798869  0.17917562 0.04877714]
 [0.29666167 0.48247987 0.09557145 0.12931094]]
[0.05119295 0.11006352 0.8834954 ]


In [207]:
# Attempt at scaling using deterministic
# EXACTLY the same results! but much slower to converge
s = mcmc.get_samples()["probs"]
print(s.std(0))
print(s.mean(0) * pdsAll / 1e-4)
print(s.mean(0) * s.sum(0) / (abs(1 - s.sum(0))))

[[0.00135276 0.00143652 0.0015077  0.00094736]
 [0.00116426 0.00074751 0.00128939 0.0008272 ]
 [0.00062613 0.0005143  0.00049686 0.00026963]
 [0.00121846 0.00132125 0.00071697 0.00086843]]
[[1214.8066    618.3147    619.5247     48.915443]
 [1740.3151    168.90504   865.8255     61.037106]
 [3542.816     319.43088   318.16068    22.740835]
 [1759.8707    859.7818    168.8585     61.09671 ]]
[[0.20509245 0.3472591  0.3479367  0.10373226]
 [0.29337636 0.09559751 0.4858643  0.12918577]
 [0.59619474 0.1798869  0.17917562 0.04877714]
 [0.29666167 0.48247987 0.09557145 0.12931094]]


In [152]:
# No scaling by pDs
s = mcmc.get_samples()["probs"]
print(s.std(0))
print(s.mean(0))

[[0.00135276 0.00143652 0.0015077  0.00094736]
 [0.00116426 0.00074751 0.00128939 0.0008272 ]
 [0.00062613 0.0005143  0.00049686 0.00026963]
 [0.00121846 0.00132125 0.00071697 0.00086843]]
[[0.20408753 0.3462562  0.3469338  0.10272242]
 [0.2923729  0.09458682 0.48486227 0.12817791]
 [0.595193   0.17888129 0.17816997 0.04775575]
 [0.29565826 0.4814778  0.09456076 0.12830308]]


In [134]:
s = mcmc.get_samples()["probs"]
print(s.std(0))
print(s.mean(0))

[[0.00112425 0.00072443 0.00122722 0.0008287 ]
 [0.00129735 0.00151242 0.001484   0.00094055]
 [0.00062192 0.00050019 0.00049377 0.00026432]
 [0.00126624 0.00133723 0.00076884 0.00085534]]
[[0.29231083 0.0946319  0.48484075 0.12821656]
 [0.20410876 0.34625798 0.34691718 0.10271599]
 [0.5951638  0.17888646 0.17819968 0.04774999]
 [0.2955718  0.48146    0.09461594 0.12835221]]


In [229]:
s = mcmc.get_samples()["probs"]
print(s.std(0))
print(s.mean(0) / (1e-4 * pdsAll))

[[0.00135276 0.00143652 0.0015077  0.00094736]
 [0.00116426 0.00074751 0.00128939 0.0008272 ]
 [0.00062613 0.0005143  0.00049686 0.00026963]
 [0.00121846 0.00132125 0.00071697 0.00086843]]
[[ 3428.6704 19390.348  19428.293  21571.709 ]
 [ 4911.8647  5296.862  27152.287  26917.361 ]
 [ 9999.243  10017.353   9977.519  10028.708 ]
 [ 4967.0586 26962.758   5295.403  26943.648 ]]


In [216]:
s.mean(0) / pdsAll 

DeviceArray([[0.34286705, 1.9390347 , 1.9428293 , 2.1571708 ],
             [0.4911865 , 0.52968615, 2.7152286 , 2.691736  ],
             [0.9999243 , 1.0017352 , 0.9977518 , 1.0028708 ],
             [0.4967059 , 2.6962757 , 0.52954024, 2.6943645 ]],            dtype=float32)

In [9]:
(liabData55cov["afs"][liabData55cov["unaffectedGenes"]]).mean(0) / liabParams55cov["afMean"]# PD|V

tensor([0.5956, 0.1787, 0.1787, 0.0476], dtype=torch.float64)

In [10]:
liabData55cov["afs"][liabData55cov["unaffectedGenes"]].mean(0).sum()

tensor(0.0001, dtype=torch.float64)

In [11]:
(liabData55cov["afs"][liabData55cov["affectedGenes"][0]]).mean(0) / liabParams55cov["afMean"]# PD|V

tensor([0.5732, 0.5439, 0.1782, 0.1450], dtype=torch.float64)

In [12]:
(liabData55cov["afs"][liabData55cov["affectedGenes"][0]]).mean(0) / (tensor(pdsAllnp) * liabParams55cov["afMean"]) #rr

tensor([0.9630, 3.0458, 0.9981, 3.0458], dtype=torch.float64)

In [13]:
(liabData55cov["afs"][liabData55cov["affectedGenes"][1]]).mean(0) / liabParams55cov["afMean"]# PD|V

tensor([0.5836, 0.1783, 0.3649, 0.0973], dtype=torch.float64)

In [12]:
pdv = (liabData55cov["afs"][liabData55cov["unaffectedGenes"]]).mean(0) / liabParams55cov["afMean"]# PD|V
print("pdv", pdv)
# 
# This matches our DM results perfectly; so the DM result matches scale
# how to allow our pdv values to work ok...
print("pdv scaled", pdv/pdv.sum())

pdv tensor([0.5962, 0.1788, 0.1788, 0.0477], dtype=torch.float64)
pdv scaled tensor([0.5952, 0.1786, 0.1786, 0.0476], dtype=torch.float64)


In [10]:
pdv = (liabData55cov["afs"][liabData55cov["affectedGenes"][0]]).mean(0) / liabParams55cov["afMean"]# PD|V
print("pdv", pdv)

# This matches our DM results perfectly; so the DM result matches scale
# how to allow our pdv values to work ok...
print("pdv scaled", pdv/pdv.sum())

pdv.sum()

pdv tensor([0.5771, 0.5481, 0.1795, 0.1462], dtype=torch.float64)
pdv scaled tensor([0.3978, 0.3778, 0.1237, 0.1007], dtype=torch.float64)


tensor(1.4508, dtype=torch.float64)

In [9]:
pdv = (liabData55cov["afs"][liabData55cov["affectedGenes"][1]]).mean(0) / liabParams55cov["afMean"]# PD|V
print("pdv", pdv)

# This matches our DM results perfectly; so the DM result matches scale
# how to allow our pdv values to work ok...
print("pdv scaled", pdv/pdv.sum())

pdv.sum()

pdv tensor([0.5834, 0.1782, 0.3651, 0.0974], dtype=torch.float64)
pdv scaled tensor([0.4766, 0.1456, 0.2983, 0.0795], dtype=torch.float64)


tensor(1.2242, dtype=torch.float64)

In [20]:
pdv = (liabData55cov["afs"][liabData55cov["affectedGenes"][2]]).mean(0) / liabParams55cov["afMean"]# PD|V
print("pdv", pdv)

# This matches our DM results perfectly; so the DM result matches scale
# how to allow our pdv values to work ok...
print("pdv scaled", pdv/pdv.sum())

pdv.sum()

pdv tensor([0.5751, 0.5526, 0.3690, 0.1229], dtype=torch.float64)
pdv scaled tensor([0.3551, 0.3412, 0.2278, 0.0759], dtype=torch.float64)


tensor(1.6196, dtype=torch.float64)

In [ ]:
# [[0.01911611 0.01343591 0.01555622 0.00563713]
#  [0.00406165 0.00276914 0.00327851 0.00133072]
#  [0.01090826 0.00992517 0.0113952  0.00424665]
#  [0.01858757 0.02309512 0.02210252 0.01013227]]
# [[0.47631553 0.10743626 0.26105177 0.03134572]
#  [0.60844487 0.14429386 0.14502415 0.01706672]
#  [0.40694135 0.31180432 0.09986749 0.0380971 ]
#  [0.35437214 0.26981634 0.2043957  0.02945131]]
# conc std [ 12.166306  71.86332   67.61688  157.65123 ]
# conc mean [ 36.537464 213.60443  190.46646  793.18304 ]
# DIrichlet version [0.02961397 0.17312849 0.15437493 0.64288265]
# inferred stick-breaking weights [0.11452598 0.665086   0.13667543 0.08672265]

In [6]:
pdvpv = (liabData55cov["afs"][liabData55cov["affectedGenes"][2]]).mean(0) #/ liabParams55cov["afMean"]# PD|V
pdv = pdvpv / liabParams55cov["afMean"]
print("pdv", pdv)
print("pdvpv", pdvpv)
print("pdvpv.sum()", pdvpv.sum())
# This matches our DM results perfectly; so the DM result matches scale
# how to allow our pdv values to work ok...
print("pdvpv scaled", pdvpv/pdvpv.sum())


pdv tensor([0.5688, 0.5427, 0.3629, 0.1207], dtype=torch.float64)
pdvpv tensor([5.6883e-05, 5.4270e-05, 3.6289e-05, 1.2074e-05], dtype=torch.float64)
pdvpv.sum() tensor(0.0002, dtype=torch.float64)
pdvpv scaled tensor([0.3566, 0.3402, 0.2275, 0.0757], dtype=torch.float64)


In [103]:
liabData55cov["altCounts"][liabData55cov["affectedGenes"][2]]

tensor([[1., 3., 5., 1.],
        [7., 5., 1., 2.],
        [6., 3., 0., 0.],
        ...,
        [7., 1., 4., 0.],
        [7., 1., 4., 1.],
        [5., 3., 4., 0.]], dtype=torch.float64)

In [92]:
pdv.sum()

tensor(1.2323, dtype=torch.float64)

In [93]:
(liabData55cov["afs"][liabData55cov["affectedGenes"][2]]).mean(0) 

tensor([5.7766e-05, 3.6280e-05, 3.6217e-05, 1.0090e-05], dtype=torch.float64)

In [94]:
liabData55cov["afs"][liabData55cov["affectedGenes"][0]].mean(0) 

tensor([5.8707e-05, 3.6776e-05, 1.7935e-05, 9.8070e-06], dtype=torch.float64)

In [95]:
pdsAllnp

[0.5952380895614624, 0.17857143, 0.17857143, 0.04761905]

In [96]:
liabParams55cov["afMean"]

tensor(1.0000e-04)

In [97]:
liabData55cov["afs"][liabData55cov["affectedGenes"][0]].mean(0)

tensor([5.8707e-05, 3.6776e-05, 1.7935e-05, 9.8070e-06], dtype=torch.float64)

In [98]:
liabParams55cov["afMean"]

tensor(1.0000e-04)

In [ ]:
# (1000, 4, 4)
# [[0.00514663 0.00313207 0.00355292 0.00149964]
#  [0.01928454 0.04950465 0.04956334 0.0065191 ]
#  [0.02503735 0.04772532 0.03840724 0.00905628]
#  [0.06047119 0.05520838 0.05940869 0.03415463]]
# [[0.607701   0.1456876  0.14313476 0.01758165]
#  [0.45891437 0.14094748 0.24490298 0.03002501]
#  [0.45052725 0.23326729 0.15602364 0.03166293]
#  [0.4925649  0.1685785  0.18608962 0.03147438]]
# conc std [18.310623   7.338482   7.672899   2.7556536]
# conc mean [46.069946 18.652845 19.728569  6.812487]
# DIrichlet version [0.50479954 0.20438373 0.2161707  0.07464606]
# inferred stick-breaking weights [0.6861298  0.14098327 0.1566307  0.01916244]